# Charity Register Data Scrape
## Louis Othen
The purpose of this script is to scrape of all charities found on the Charity register owned by the governments Charity Commission. the register can be found here: 
https://www.gov.uk/find-charity-information

The reason for the scrape is to collect information on a charity, what it does, what contact information it has, and format into a table. This table will be joined with another previous created before sending a mass email out to them to offer my volunteering services in data.

# Import the relevant libraries

In [1]:
import pandas as pd

from selenium                       import webdriver
from selenium.webdriver.common.by   import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from tqdm import tqdm 
import requests as req
import time as tim
from lxml import etree
#from ydata_profiling import ProfileReport  #Stopped working since python 3.12
import re
from plotly import express as px
from datetime import datetime as dti

## Inital Scrape

### Set up with selenium to get past cookies

In [37]:
# Open up test browser window
#-----------------------------------------------------
ded = webdriver.Chrome()

# Navigate to charity register screen 
#-----------------------------------------------------
ded.get('https://register-of-charities.charitycommission.gov.uk/charity-search/-/results/page/1/delta/20/sorted-by/charity-income/desc')

# Accept cookies
#-----------------------------------------------------
cookies_btn = ded.find_element(By.ID,'_com_placecube_cookieconsent_web_portlet_CookieConsentPortlet_acceptAllCookies')
cookies_btn.click()

In [38]:
# Get list of page numbers
#-----------------------------------------------------
end_page_len = ded.find_elements(By.ID, "_uk_gov_ccew_portlet_CharitySearchPortlet_pageIterator")

# Get value of last page number 
#-----------------------------------------------------
pe = end_page_len[0].text.split(sep='\n')
end_page_num = int(pe[-2])

### Using requests to parse the contents of each page

In [ ]:
header = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        }

# Empty list ready to store looped data 
#-----------------------------------------------------
collected_data  = []

# For loop over webpages, to download high level charity data
#-----------------------------------------------------
for i in tqdm(range(1,end_page_num)):
    url_use = 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/results/page/' + str(i) + '/delta/20/sorted-by/charity-income/desc'#
    print(url_use)
    r= req.get(url_use, headers = header)
    collected_data.append(pd.read_html(r.text)[0])

# Save collected list as a pandas DataFrame
#-----------------------------------------------------
result = pd.concat(collected_data)

# Remove unneeded columns
#-----------------------------------------------------
result.drop(columns=['level_0', 'index'], inplace=True)
    

In [ ]:
# See results of scrape before saving down into CSV file.
#-----------------------------------------------------
#result = pd.concat(collected_data)
display(result)
result.to_csv('D:\Coding\Charity_Scrape\charityregister.csv')

## Now to get the child data from each individual link  

### Set up dataframe with columns to mine from webpages

In [ ]:
# Read in previously farmed dataframe of summary pages
#-----------------------------------------------------
result = pd.read_csv('D:\Coding\Charity_Scrape\charityregister.csv')

# Remove records with Nulls - not useful at this stage - may be worth looking at later
#-----------------------------------------------------
result = result.dropna()

result.drop(columns = ['Unnamed: 0'], inplace = True) # weird index as per previous download -  remove

# Set up columns ready for further mining
#-----------------------------------------------------
result['Charity number'] = result['Charity number'].astype(str).str.split('.').str[0]
result['overview'] = ''
result['what_it_does'] = ''
result['who_it_benefits'] = ''
result['how_its_done'] = ''
result['where_they_operate'] = ''
result['address'] = ''
result['phone'] = ''
result['email'] = ''
result['website'] = ''

# get a sample to play with
testo = result


In [ ]:
# function to automate 1) get link and content , and prep for xpath extraction
#-----------------------------------------------------
def get_page(url):
    header =    {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
                }
    try:
        
        # page = req.session()
        # page.cookies.update({'__hs_opt_out': 'no'})

        url = url
        charity_page = req.get(url=url_use, headers=header)

        soup = bs(charity_page.text,'lxml')
        dom = etree.HTML(str(soup))
    
        return dom
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return None

In [ ]:
# Loop through each charity details and store it 

for index, row in tqdm(testo.iterrows(), total = testo.shape[0]):
     url_use = 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/charity-details/' + row['Charity number']

     xpage = get_page(url_use)
     try:
          testo.loc[index, 'overview']            = xpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[1]/div/p')[0].text
     except IndexError:
          testo.loc[index, 'overview']            = "N/A"

     url_use = url_use + '/what-who-how-where'
     wpage = get_page(url_use)

     try:
          testo.loc[index, 'what_it_does']        = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[2]/div[2]/ul/li')[0].text
          testo.loc[index, 'who_it_benefits']     = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[3]/div[2]/ul/li')[0].text
          testo.at[index, 'how_its_done']         = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[4]/div[2]/ul/li/text()')
          testo.at[index, 'where_they_operate']   = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[5]/div[2]/ul/li/text()')
     except IndexError:
          testo.loc[index, 'what_it_does']        = "N/A"
          testo.loc[index, 'who_it_benefits']     = "N/A"
          testo.loc[index, 'how_its_done']        = "N/A"
          testo.loc[index, 'where_they_operate']  = "N/A"
     url_use = url_use.replace('/what-who-how-where', '')

     url_use = url_use + '/contact-information'
     cpage = get_page(url_use)


     try:
          testo.at[index, 'address']              = cpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/dl/div[1]/dd/text()')
          testo.loc[index, 'phone']               = cpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/dl/div[2]/dd/text()')
                                                                 
          testo.loc[index, 'email']               = cpage.xpath('//*[@id="charity-contact-email-link"]/text()')
          testo.loc[index, 'website']             = cpage.xpath('//*[@id="charity-contact-website-link"]/text()')
     except Exception:
          testo.loc[index, 'address']             = "N/A"
          testo.loc[index, 'phone']               = "N/A"
          testo.loc[index, 'email']               = "N/A"
          testo.loc[index, 'website']             = "N/A"
     
     tim.sleep(0.05)

     

     

Save this Data!

In [ ]:
testo.to_csv('D:\Coding\Charity_Scrape\charityregister_detail_raw.csv')

## Data Cleaning
now that all the data we need has been farmed / mined, next phase is to clean things up before performing Exploratory data analysis upon it

 bring the data back in 

In [69]:
raw_data = pd.read_csv("D:\Coding\Charity_Scrape\charityregister_detail_raw.csv")

clean = raw_data
clean.dropna(
                            subset = [
                                        'overview'
                                        ,'what_it_does'
                                        ,'who_it_benefits'
                                        ,'how_its_done'
                                        ,'where_they_operate'
                                        ,'address'
                                        ,'phone'
                                        ,'email'
                                        ,'website'
                                     ]
                           ,inplace= True
                        )
clean.reset_index(inplace = True)
clean.drop(columns=['Unnamed: 0', 'Reporting', 'Status','how_its_done','where_they_operate','Charity number','index'], inplace=True)

C:\Users\lothe\AppData\Local\Temp\ipykernel_18724\700711209.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("D:\Coding\Automation\charityregister_detail_raw.csv")


In [78]:
clean['overview'] = clean['overview'].str.replace('\n', '').str.replace('\t', '').str.replace("'",'')

# Removed or now as messy from mining, but could use later
#--------------------------------------------------------------------------------------
#clean['how_its_done'] = clean['how_its_done'].str.replace('[', '').str.replace("'",'')
#clean['how_its_done'] = clean['how_its_done'].str.replace(']', '').str.replace("'",'')    
#clean['where_they_operate'] = clean['where_they_operate'].str.replace('[', '').str.replace(']', '').str.replace("'",'')

clean['website'] = clean['website'].str.replace('[','').str.replace(']', '').str.replace("'",'')
clean['email'] = clean['email'].str.replace('[','').str.replace(']', '').str.replace("'",'')
clean['phone'] = clean['phone'].apply(lambda x: re.findall(r'\d+', str(x))).apply(lambda x: ''.join(map(str,x)))
clean['address'] = clean['address'].apply(lambda x: re.findall(r'\w+\d*\s+',str(x))).apply(lambda x: [item.replace('t ', '')  if item == 't ' else item for item in x ]).apply(lambda x: ' '.join(x)).apply(lambda x: re.sub(r'\s+', ' ',x).strip())

clean['Income'] = clean['Income'].str.replace('£','').str.replace(',','')

clean = clean.applymap(lambda x: x.lower())

In [79]:
clean.head()

,Charity name,Income,overview,what_it_does,who_it_benefits,address,phone,email,website
0,the charities aid foundation,1044275000,charities aid foundation (caf) is a registered...,general charitable purposes,other charities or voluntary bodies,charities aid foundation 25 kings hill avenue ...,03000123088,companysecretary@cafonline.org,www.cafonline.org
1,nuffield health,995600000,"to advance, promote and maintain health and he...",children/young people,provides services,nuffield health epsom gateway 2 ashley avenue ...,07824482211,iben.thomson@nuffieldhealth.com,www.nuffieldhealth.com
2,the arts council of england,941845043,arts council england works to get great art to...,arts/culture/heritage/science,other charities or voluntary bodies,arts council england the hive 49 lever street ...,01619344317,enquiries@artscouncil.org.uk,www.artscouncil.org.uk
3,the british council,896655974,the british council creates friendly knowledge...,the general public/mankind,makes grants to individuals,1 redman place stratford london united kingdom,01619577755,ceo.office@britishcouncil.org,www.britishcouncil.org
4,the national trust for places of historic inte...,643329000,to look after places of historic interest or n...,the general public/mankind,provides buildings/facilities/open space,national trust heelis kemble drive swindon,01793817400,thesecretary@nationaltrust.org.uk,www.nationaltrust.org.uk


In [42]:
# Currently not working since updating python to 3.12
#ProfileReport(clean)

# Lets save it for more stuff later ! 


In [80]:
clean.to_csv("D:\Coding\Charity_Scrape\charityregister_detail_clean.csv")

# Read it back in for analysis of what we want to go for 

In [8]:
data = pd.read_csv("D:\Coding\Charity_Scrape\charityregister_detail_clean.csv")

In [9]:
data.head()
data.drop(columns=['Unnamed: 0'], inplace=True)

bring previous charity data and join them together


In [11]:
prev_data = pd.read_csv("D:\Coding\Charity_Scrape\charity_email_list.csv")
final_data = pd.concat([data, prev_data])
final_data.columns

Index(['Charity name', 'Income', 'overview', 'what_it_does', 'who_it_benefits',
       'address', 'phone', 'email', 'website'],
      dtype='object')

In [13]:
final_data

,Charity name,Income,overview,what_it_does,email,website
0,the charities aid foundation,1044275000,charities aid foundation (caf) is a registered...,general charitable purposes,companysecretary@cafonline.org,www.cafonline.org
1,nuffield health,995600000,"to advance, promote and maintain health and he...",children/young people,iben.thomson@nuffieldhealth.com,www.nuffieldhealth.com
2,the arts council of england,941845043,arts council england works to get great art to...,arts/culture/heritage/science,enquiries@artscouncil.org.uk,www.artscouncil.org.uk
3,the british council,896655974,the british council creates friendly knowledge...,the general public/mankind,ceo.office@britishcouncil.org,www.britishcouncil.org
4,the national trust for places of historic inte...,643329000,to look after places of historic interest or n...,the general public/mankind,thesecretary@nationaltrust.org.uk,www.nationaltrust.org.uk
...,...,...,...,...,...,...
13,Macmillan,0,cancer support,cancer support,contact@macmillan.org.uk,https://www.macmillan.org.uk/
14,Revitalise,0,repsirte for carer and disabled,repsirte for carer and disabled,info@revitalise.org.uk,https://revitalise.org.uk
15,The Earl of Southampton Trust,0,local housing for the elderly,local housing for the elderly,info@eost.org.uk,https://eost.org.uk
16,Dogs Trust,0,Dog Rehoming,Dog Rehoming,info@dogstrust.org.uk,https://www.dogstrust.org.uk


In [16]:
final_data.to_csv("D:\Coding\Charity_Scrape\charity_email_merged.csv")

# Analyse what is there and remove unwanted

In [17]:
df = pd.read_csv("D:\Coding\Charity_Scrape\charity_email_merged.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns
df.shape

(12144, 6)

Get rid of more charities within the category chosen, such as religious or partisan related. Essentially, avoid anything that could be contrived as 'controversial'.


In [25]:
words_to_remove = ['islam', 'islamic','diocese','synagogue','ARTHRITIS','HORSE','CHURCH','GOD','Religion','Catholic','MOTORISTS','Christian'
                   ,'SPIRITUAL','COAL','Gospel','THEATRE','Missionary','VEGAN','CINEMA','evange','football','DANCE','choir'
                   ,'estate','trains','Holocaust','ROMAN','SPORTS','GALLERY','opera','CLUB','POLISH','MANOR','LEISURE','worship'
                   ,'chapel','cats','BUDDHIST','League','Sisters','museum','sheep','breeders','BOTANICAL','NAUTICAL', 'BUILDINGS'
                   ,'SIKH','chinese','ART','AGRICULTURAL','HALL','DIOCESAN','PRODUCTIONS','COLLECTION','PALACE'
                   ,'ORCHESTRA','WHALE', 'DOLPHIN','Christian','BIBLE','CHRIST','theological','Jesus','festival']

cols_to_search = ['Charity name']

row_rm = c_data[cols_to_search].apply(lambda x: x.str.contains('|'.join(words_to_remove), case=False, na=False)).any(axis=1)

c_data = c_data[~row_rm]

c_data.shape

(9231, 6)

In [54]:
# Save final filtered data ready to choose charities to approach
#-----------------------------------------------------
c_data.to_csv("D:\Coding\Automation\charity_email_final.csv")

# Now to look at charities with a particular condition to focus on.


In [3]:
c_data = pd.read_csv("D:/Coding/git_projects/Charity_Scrape/charity_email_final.csv")
display(c_data)

,Unnamed: 0.1,Unnamed: 0,Charity name,Income,overview,what_it_does,email,website
0,6456,8214,the gibbs charitable trusts,100001,only uk registered charities supported. prospe...,general charitable purposes,jamesgibbs@btinternet.com,www.gibbstrust.org.uk
1,6455,8213,the morris charitable trust,100008,the morris charitable trust has been establish...,general charitable purposes,info@morrischaritabletrust.com,www.morrischaritabletrust.com
2,6454,8212,refugee women's association,100033,we provide iag services to refugee/migrant wom...,other defined groups,rwa@refugeewomen.org.uk,www.refugeewomen.org.uk
3,6453,8211,the hebridean trust limited,100077,the hebridean trust was formed in 1982 with th...,children/young people,info@hebrideantrust.org,www.hebrideantrust.org
4,6452,8209,the royal archaeological institute,100142,runs a lecture programme from october to may; ...,other defined groups,admin@royalarchinst.org,www.royalarchinst.org
...,...,...,...,...,...,...,...,...
6382,4,59,consumers' association,86673000,we aim to empower consumers to make informed c...,the general public/mankind,which@which.co.uk,www.which.co.uk
6383,3,58,education development trust,89070000,the charity seeks to advance education for the...,children/young people,corpgov@educationdevelopmenttrust.com,http://www.educationdevelopmenttrust.com/
6384,2,57,wateraid,91913000,"wateraid is an international not-for-profit, d...",the prevention or relief of poverty,supportercare@wateraid.org,www.wateraid.org
6385,1,56,the halo trust,94925000,halo\r1. helps to eliminate the threat to live...,the general public/mankind,mail@halotrust.org,www.halotrust.org


In [4]:
c_types = set(c_data['what_it_does'])
c_types

{'accommodation/housing',
 'amateur sport',
 'animals',
 'armed forces/emergency service efficiency',
 'arts/culture/heritage/science',
 'children/young people',
 'disability',
 'economic/community development/employment',
 'education/training',
 'elderly/old people',
 'environment/conservation/heritage',
 'general charitable purposes',
 'human rights/religious or racial harmony/equality or diversity',
 'other charitable purposes',
 'other charities or voluntary bodies',
 'other defined groups',
 'overseas aid/famine relief',
 'people of a particular ethnic or racial origin',
 'people with disabilities',
 'recreation',
 'religious activities',
 'the advancement of health or saving of lives',
 'the general public/mankind',
 'the prevention or relief of poverty'}

In [8]:
c_types_pick1 = ['general charitable purposes']
c_types_pick2 = ['the general public/mankind']

to_send = c_data[(c_data['what_it_does'].isin(c_types_pick2))]

to_send

,Unnamed: 0.1,Unnamed: 0,Charity name,Income,overview,what_it_does,email,website
10,6446,8199,london topographical society,100408,the london topographical society concentrates ...,the general public/mankind,mike.wicksteed@btinternet.com,www.londontopsoc.org
19,6437,8186,royal society of painter-printmakers,100657,"exhibitions, lectures, demonstrations",the general public/mankind,info@banksidegallery.com,www.re-printmakers.com
34,6422,8165,hetton town trust,101273,we are a community facility in which we run co...,the general public/mankind,kim@hettontowntrust.co.uk,www.hettontowntrust.co.uk
39,6417,8155,guild of one-name studies,101516,advance the education of the public in one-nam...,the general public/mankind,guild@one-name.org,www.one-name.org
70,6386,8120,the solway aviation society limited,102500,to preserve for the general public an aviation...,the general public/mankind,info@solway-aviation-museum.co.uk,www.solway-aviation-museum.co.uk
...,...,...,...,...,...,...,...,...
6357,29,87,care international uk,62719000,care international is a global charity dedicat...,the general public/mankind,info@careinternational.org,www.careinternational.org.uk
6374,12,69,the woodland trust,76376000,our objects are to conserve restore & re-estab...,the general public/mankind,enquiries@woodlandtrust.org.uk,www.woodlandtrust.org.uk
6377,9,65,the zoological society of london,79510000,to achieve and promote the conservation of ani...,the general public/mankind,info@zsl.org,www.zsl.org
6382,4,59,consumers' association,86673000,we aim to empower consumers to make informed c...,the general public/mankind,which@which.co.uk,www.which.co.uk


#  Compile final list to email and dated ( so to track who i emailed) 

In [9]:
charity_filepath = 'D:\\Coding\\git_projects\\Charity_Scrape\\' + 'charity_email_sent_' + dti.now().strftime("%Y-%m-%d") + '.csv'
print(charity_filepath)
to_send.to_csv(charity_filepath)

D:\Coding\git_projects\Charity_Scrape\charity_email_sent_2024-02-08.csv


In [13]:
# c_data
to_send

,Unnamed: 0.1,Unnamed: 0,Charity name,Income,overview,what_it_does,email,website
0,6456,8214,the gibbs charitable trusts,100001,only uk registered charities supported. prospe...,general charitable purposes,jamesgibbs@btinternet.com,www.gibbstrust.org.uk
1,6455,8213,the morris charitable trust,100008,the morris charitable trust has been establish...,general charitable purposes,info@morrischaritabletrust.com,www.morrischaritabletrust.com
2,6454,8212,refugee women's association,100033,we provide iag services to refugee/migrant wom...,other defined groups,rwa@refugeewomen.org.uk,www.refugeewomen.org.uk
3,6453,8211,the hebridean trust limited,100077,the hebridean trust was formed in 1982 with th...,children/young people,info@hebrideantrust.org,www.hebrideantrust.org
4,6452,8209,the royal archaeological institute,100142,runs a lecture programme from october to may; ...,other defined groups,admin@royalarchinst.org,www.royalarchinst.org
...,...,...,...,...,...,...,...,...
6382,4,59,consumers' association,86673000,we aim to empower consumers to make informed c...,the general public/mankind,which@which.co.uk,www.which.co.uk
6383,3,58,education development trust,89070000,the charity seeks to advance education for the...,children/young people,corpgov@educationdevelopmenttrust.com,http://www.educationdevelopmenttrust.com/
6384,2,57,wateraid,91913000,"wateraid is an international not-for-profit, d...",the prevention or relief of poverty,supportercare@wateraid.org,www.wateraid.org
6385,1,56,the halo trust,94925000,halo\r1. helps to eliminate the threat to live...,the general public/mankind,mail@halotrust.org,www.halotrust.org


if these are in original register, remove them

In [17]:
# to_send = pd.read_csv("D:/Coding/git_projects/Charity_Scrape/charity_email_sent_2023-12-20.csv")
# to_send.head()

# c_data.drop(labels = to_send['Unnamed: 0'], axis=0, inplace=True)
# c_data.drop(labels=to_send.index, axis=0, inplace=True)
c_data.to_csv('D:/Coding/git_projects/Charity_Scrape/charity_email_final.csv')